In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("cs_bisnode_panel.csv")

df = df[(df["year"] >= 2010) & (df["year"] <= 2015)].copy()

# Sales Data
df["sales"] = df["sales"].clip(lower=1)
df["sales_mil"] = df["sales"] / 1_000_000
df["sales_mil_log"] = np.log(df["sales_mil"])
df["sales_mil_log_sq"] = df["sales_mil_log"] ** 2

# New Company
df["age"] = (df["year"] - df["founded_year"]).clip(lower=0)
df["new"] = ((df["age"] <= 1) | (df["balsheet_notfullyear"] == 1)).astype(int)

# Increase Rate
df = df.sort_values(["comp_id", "year"])
df["d1_sales_mil_log"] = df.groupby("comp_id")["sales_mil_log"].diff().fillna(0)
df["d1_sales_mil_log_mod"] = np.where(df["new"] == 1, 0, df["d1_sales_mil_log"])

# Sales Growth
sales_pivot = df.pivot(index="comp_id", columns="year", values="sales_mil_log")
sales_pivot["sales_growth_5y"] = sales_pivot[2015] - sales_pivot[2010]
growth = sales_pivot[["sales_growth_5y"]].dropna().reset_index()

# Fast Growth
threshold = growth["sales_growth_5y"].quantile(0.8)
growth["fast_growth"] = (growth["sales_growth_5y"] >= threshold).astype(int)

df = df.merge(growth, on="comp_id", how="left")
drop_vars = [
    "address", "zip", "county_code", "founded_year", "sales", "balsheet_length",
    "balsheet_notfullyear", "comp_name", "id", "status_alive", "exit_year",
    "exit", "death_year", "death_code", "begin", "end", "COGS"
]
df.drop(columns=[col for col in drop_vars if col in df.columns], inplace=True)

X = df[df["year"] == 2014].copy()
y = df[df["year"] == 2015][["comp_id", "fast_growth"]].copy()
model_df = X.merge(y, on="comp_id", how="inner", suffixes=('', '_label'))

df.to_csv("bisnode_panel_2010_2015_full_cleaned.csv", index=False)
model_df.to_csv("bisnode_model_2014_predict_2015_cleaned.csv", index=False)